In [1]:
#pip install seaborn

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
import shap
import seaborn as sns
import plotly.express as px
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
import calendar
import requests
from sklearn.compose import ColumnTransformer

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [3]:
def read_file(file_path, year_from=1980):
    # Define the number of rows to skip
    num_rows_to_skip = 6
    # Read the Excel file, skipping the specified number of rows
    df = pd.read_excel(file_path, skiprows=num_rows_to_skip)
    
    df_temp = df[df["Disaster Type"]=="Extreme temperature "]
    df_heat = df_temp[df_temp['Disaster Subtype']=='Heat wave']
    
    df_heat_recent = df_heat[df_heat['Year']>=year_from]
    
    print("there are " + str(df_heat_recent.shape[0]) + " heatwave events")
    return df_heat_recent

In [4]:
directory = '../data/raw/'
# the earliest data available is from 1985
df_heat = read_file(directory + 'emdat_public_2023_08_22_query_uid-hU230X.xlsx')

df_heat.head()

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [ ]:
def countries_with_min_no_heatwaves(df_heat, threshold=3):
    select = df_heat.groupby('Country').count().reset_index().sort_values(by='Year', ascending=False)
    return len(select[select['Year']>=threshold])

top = countries_with_min_no_heatwaves(df_heat, threshold=3)
top

In [ ]:
def heatwave_top_countries(df, top=None):
    df_sorted = df.groupby('Country').count().reset_index().sort_values(by='Year', ascending=False)
    
    if top == None:
        return_top = len(df_sorted) 
    else:
        return_top = top
    plt.barh(df_sorted.Country, df_sorted.Year, color=['red' if country in df_sorted.iloc[:return_top]['Country'].values else 'blue' for country in df_sorted.Country])
    plt.title('No. heatwaves per country')
    
    focus = list(df_sorted.iloc[:return_top]['Country'])
    top_country_data = df[df['Country'].isin(focus)] 
    print(top_country_data.shape)
    return top_country_data

In [ ]:
top_countries = heatwave_top_countries(df_heat, top)

In [ ]:
def plot_deaths_overtime(df):
    '''
    Missing data are now considered and plotted as grey dots
    '''
    data = df.groupby('Year')["Total Deaths"].sum().reset_index()

    plt.figure(figsize=(10, 6))

    plt.plot(data['Year'], data['Total Deaths'], marker='o', linestyle='-', color='blue', label='Total Deaths')
    
    # Identify missing years and plot grey dots for them
    all_years = np.arange(data['Year'].min(), data['Year'].max() + 1)
    missing_years = np.setdiff1d(all_years, data['Year'])
    if len(missing_years) > 0:
        missing_data = np.zeros(len(missing_years))
        plt.scatter(missing_years, missing_data, color='grey', marker='o', label='Missing Years')

    plt.xlabel('Year')
    plt.ylabel('Total Deaths')
    plt.title('Line Chart with Deaths')
    
    # Set x-axis ticks to include all years
    plt.xticks(all_years)

    plt.legend()
    plt.show()

In [ ]:
plot_deaths_overtime(top_countries)

# as we have better equipment to protect from disaster the number of death can be decreasing

In [ ]:
top_countries_valid_death = top_countries[~top_countries['Total Deaths'].isna()]#.shape

In [ ]:
dfs = top_countries_valid_death.copy()
#dfs.to_csv('../data/processed/country_focus.csv')

In [ ]:
heatmap_data = dfs.pivot_table(index='Country', columns='Year', values='Total Deaths', aggfunc='sum')

# Create a heatmap using seaborn
plt.figure(figsize=(10, 6))
sns.heatmap(heatmap_data, cmap='YlGnBu', annot=False)
plt.title('Number of Deaths by Country and Year')
plt.xlabel('Year')
plt.ylabel('Country')
plt.show()

In [ ]:
def world_map(df, year):
    # Erstellung der Karte
    # Standorte = Alpha-3-Code für jedes Land
    
    df = df[df['Year']==year]
    print(df.shape)
    
    fig = px.choropleth(df, locations='ISO', color='Total Deaths',
                        color_continuous_scale='RdYlGn_r', template="plotly_dark",
                        title="Worldwide Death Counts in " + str(year))
    
    # setzen der Mapbox
    # Stelle den Zoom ein, wenn das Diagramm angezeigt wird
    fig.update_layout(mapbox_style="carto-positron",
                      mapbox_zoom=10)
    # Titel
    fig.update_layout(title_x=0.5, margin={"r":0, "t":30,"l":0,"b":0})
    
    # Display the figure
    fig.show()

# Plot for particular year
world_map(dfs, 2003)

In [ ]:
def europe_map(df, year):
    fig = px.choropleth(df, locations='ISO', color='Total Deaths',
                        color_continuous_scale='RdYlGn_r', template="plotly_dark",
                        title="European Death Counts in " + str(year))

    # Define the boundary coordinates for Europe
    europe_boundary = {
        "type": "Feature",
        "geometry": {
            "type": "Polygon",
            "coordinates": [
                [
                    [-20, 35],
                    [40, 35],
                    [40, 70],
                    [-20, 70],
                    [-20, 35]
                ]
            ]
        }
    }

    # Add boundary shape to the map
    fig.add_shape(type="line",
                  x0=-20, y0=35, x1=40, y1=35,
                  line=dict(color="blue", width=2))

    fig.update_geos(
        fitbounds="locations",
        visible=False,  # Hide the world map
        showcoastlines=False,  # Hide coastlines
        projection_scale=1,  # Adjust scale if needed
        center={"lat": 52.0, "lon": 10.0},  # Adjust center as needed
        scope="europe",  # Set the map scope to Europe
    )

    fig.update_layout(title_x=0.5, margin={"r": 0, "t": 30, "l": 0, "b": 0})

    fig.show()

# Call the function
europe_map(dfs, 2003)